In [2]:
import sqlite3
import os
from contextlib import contextmanager 
from datetime import date

In [3]:
BACKEND_DIR = '/Users/dilshantharushika/Desktop/laptop agent/backend' 
DB_NAME = 'laptops_dynamic.db'
DB_PATH = os.path.join(BACKEND_DIR, DB_NAME)

print("Configuration loaded.")
print(f"Database path set to: {os.path.abspath(DB_PATH)}")

Configuration loaded.
Database path set to: /Users/dilshantharushika/Desktop/laptop agent/backend/laptops_dynamic.db


In [4]:
if not os.path.exists(DB_PATH):
    print(f"\n WARNING ")
    print(f"Database file not found at '{DB_PATH}'!")
    print("Please run the 'setup_dynamic_db.ipynb' notebook first to create and seed the database.")
    
else:
    print("Database file found.")

Database file found.


In [5]:
@contextmanager
def get_db_connection():
    """Provides a managed database connection to the dynamic DB."""
    conn = None
    try:
        if not os.path.exists(DB_PATH):
             raise FileNotFoundError(f"Database file not found at {DB_PATH}. Cannot connect.")
        conn = sqlite3.connect(DB_PATH)
        conn.row_factory = sqlite3.Row 
        
        yield conn
    except Exception as e:
        print(f"Database connection error: {e}")
        yield None 
    finally:
        if conn:
            conn.close()
           

print("Database helper function `get_db_connection` defined.")

Database helper function `get_db_connection` defined.


In [6]:
def get_all_laptops():
    """Fetches all laptops from the Laptop table."""
    laptops = []
    try:
        with get_db_connection() as conn:
            if conn is None: return [] 
            cursor = conn.cursor()
            cursor.execute("SELECT sku, brand, model_name, currency, availability, shipping_eta, review_count, average_rating FROM Laptop")
            rows = cursor.fetchall()
          
            laptops = [dict(row) for row in rows]
            print(f"Successfully fetched {len(laptops)} laptops from the catalog.")
    except Exception as e:
        print(f"Error fetching laptop catalog: {e}")
    return laptops

print("`get_all_laptops` function defined.")

`get_all_laptops` function defined.


In [7]:
def get_price_history_for_sku(sku):
    """Fetches price history for a specific SKU, ordered by date."""
    prices = []
    try:
        with get_db_connection() as conn:
            if conn is None: return []
            cursor = conn.cursor()
            cursor.execute("""
                SELECT id, price, date, vendor_name, promo_badges
                FROM PriceHistory
                WHERE laptop_sku = ?
                ORDER BY date DESC
            """, (sku,))
            rows = cursor.fetchall()
            prices = [dict(row) for row in rows]
            print(f"Found {len(prices)} price records for SKU '{sku}'.")
    except Exception as e:
        print(f"Error fetching price history for SKU '{sku}': {e}")
    return prices

def get_reviews_for_sku(sku):
    """Fetches reviews for a specific SKU, ordered by date."""
    reviews = []
    try:
        with get_db_connection() as conn:
            if conn is None: return []
            cursor = conn.cursor()
            cursor.execute("""
                SELECT id, rating, review_text, date, source
                FROM Review
                WHERE laptop_sku = ?
                ORDER BY date DESC
            """, (sku,))
            rows = cursor.fetchall()
            reviews = [dict(row) for row in rows]
            print(f"Found {len(reviews)} reviews for SKU '{sku}'.")
    except Exception as e:
        print(f"Error fetching reviews for SKU '{sku}': {e}")
    return reviews

def get_qanda_for_sku(sku):
    """Fetches Q&A for a specific SKU, ordered by date."""
    qanda = []
    try:
        with get_db_connection() as conn:
            if conn is None: return []
            cursor = conn.cursor()
            cursor.execute("""
                SELECT id, question_text, answer_text, date, source
                FROM QuestionAnswer
                WHERE laptop_sku = ?
                ORDER BY date DESC
            """, (sku,))
            rows = cursor.fetchall()
            qanda = [dict(row) for row in rows]
            print(f"Found {len(qanda)} Q&A records for SKU '{sku}'.")
    except Exception as e:
        print(f"Error fetching Q&A for SKU '{sku}': {e}")
    return qanda

print("Functions `get_price_history_for_sku`, `get_reviews_for_sku`, and `get_qanda_for_sku` defined.")

Functions `get_price_history_for_sku`, `get_reviews_for_sku`, and `get_qanda_for_sku` defined.


In [8]:

print("Testing: Fetch All Laptops ")
all_laptops_data = get_all_laptops()

if all_laptops_data:

    print("\nSample Laptop Data (first entry):")
    for key, value in all_laptops_data[0].items():
        print(f"  {key}: {value}")
else:
    print("No laptop data found. Ensure the database was seeded correctly.")

Testing: Fetch All Laptops 
Successfully fetched 4 laptops from the catalog.

Sample Laptop Data (first entry):
  sku: ThinkPad E14 Gen 5 (Intel)
  brand: Lenovo
  model_name: ThinkPad E14 Gen 5 (Intel)
  currency: LKR
  availability: In Stock
  shipping_eta: Ships in 3-5 days
  review_count: 21
  average_rating: 3.7


In [9]:

test_sku = "ThinkPad E14 Gen 5 (Intel)"


print(f"\n--- Testing: Fetching dynamic data for SKU: '{test_sku}' ---")


print("\n--- Price History ---")
price_data = get_price_history_for_sku(test_sku)
if price_data:
    for price_record in price_data[:3]: 
        print(price_record)
else:
    print("No price history found for this SKU.")


print("\n--- Reviews ---")
review_data = get_reviews_for_sku(test_sku)
if review_data:
    for review_record in review_data[:3]: # Print first 3 reviews
        print(review_record)
else:
    print("No reviews found for this SKU.")


print("\n--- Q&A ---")
qanda_data = get_qanda_for_sku(test_sku)
if qanda_data:
    for qa_record in qanda_data[:3]: 
        print(qa_record)
else:
    print("No Q&A found for this SKU.")

print("\n--- Testing complete ---")


--- Testing: Fetching dynamic data for SKU: 'ThinkPad E14 Gen 5 (Intel)' ---

--- Price History ---
Found 5 price records for SKU 'ThinkPad E14 Gen 5 (Intel)'.
{'id': 4, 'price': 449099.34, 'date': '2025-10-06', 'vendor_name': 'Partner Reseller B', 'promo_badges': 'Free Shipping'}
{'id': 1, 'price': 464421.4, 'date': '2025-09-29', 'vendor_name': 'Official Store', 'promo_badges': 'None'}
{'id': 3, 'price': 453970.6, 'date': '2025-09-20', 'vendor_name': 'Partner Reseller B', 'promo_badges': 'Sale'}

--- Reviews ---
Found 3 reviews for SKU 'ThinkPad E14 Gen 5 (Intel)'.
{'id': 3, 'rating': 4, 'review_text': 'Display is clear and bright.', 'date': '2025-08-21', 'source': 'Tech Site'}
{'id': 2, 'rating': 5, 'review_text': "Battery doesn't last a full day.", 'date': '2025-06-05', 'source': 'Customer Review'}
{'id': 1, 'rating': 3, 'review_text': 'Decent value for money.', 'date': '2025-05-07', 'source': 'Tech Site'}

--- Q&A ---
Found 2 Q&A records for SKU 'ThinkPad E14 Gen 5 (Intel)'.
{'id'